In [1]:
from __future__ import division, print_function

# to suppress warnings of Seaborn's deprecated usage of Matplotlib
import warnings
warnings.filterwarnings("ignore")

<center>
<h1>Handling GPS data with Python</h1>
<br>
<h3>Dr. Florian Wilhelm</h3>
<h4>Senior Data Scientist @ inovex</h4>
</center>

<center><h1>Jupyter/IPython Notebook</h1>
<img src="./pics/inovex_logo.svg" widht=60%><br style="clear:both"/>
Live demonstration
</center>

<center><h1>Titanic: Analysis of a Disaster</h1>
<img src="./gfx/inovex_logo.svg" widht=60%><br style="clear:both"/>
Painting from <a href=https://en.wikipedia.org/wiki/Sinking_of_the_RMS_Titanic>Willy Stöwer</a>, source & more information: <a href=https://www.kaggle.com/c/titanic>Kaggle</a>
</center>

<center>
    <h1>Example with controls and no auto-play</h1>
    <img id="example1" src="./gfx/rdp_animation_large.gif" rel:animated_src="./gfx/rdp_animation_large.gif" rel:auto_play="0" width="1024" height="1768" />
    <br>
    <script type="text/javascript">
        var sup1 = requirejs(["js/libgif"], function(SuperGif){
            var sup1 = new SuperGif({ gif: document.getElementById('example1') } );
            sup1.load();
            document.getElementById('button_pause').onmousedown = function(event) {sup1.pause(); return false;};
            document.getElementById('button_play').onmousedown = function(event) {sup1.play(); return false;};
            document.getElementById('button_restart').onmousedown = function(event) {sup1.move_to(0); return false;};
            document.getElementById('button_forward').onmousedown = function(event) {sup1.move_relative(1); return false;};
            document.getElementById('button_backword').onmousedown = function(event) {sup1.move_relative(-1); return false;};
        
        })
    </script>
    <a href="javascript:;" id="button_pause">Pause</a> |
    <a href="javascript:;" id="button_play">Play</a> |
    <a href="javascript:;" id="button_restart">Restart</a> |
    <a href="javascript:;" id="button_forward">Step forward</a> |
    <a href="javascript:;" id="button_backword">Step back</a>
</center>

<center><h1>Can we predict who survived on the Titanic?</h1>
<p><p>
<font size="5">Based on the properties of a passenger like:
<ul>
  <li>gender,</li>
  <li>age,</li>
  <li>passenger class,</li>
  <li>number of siblings,</li>
  <li>...</li>
</ul>
</font>
</p>
</center>

## Additional libraries for Data Science

#### Visualisation
* **Bokeh** allows interactive visualization like plots, dashboards in web browsers.
* **Pygal** for standard but sexy charts in SVG.

#### High-performance computing
* **Theano** evaluate mathematical expressions involving multi-dimensional arrays efficiently.
* **Spark** is fast and general-purpose cluster computing system with PySpark as Python interface.
* **Cython** is an optimising static compiler for writing C-extensions in a Python-like language.
* **Dask** enables parallel computing through task scheduling and blocked algorithms (Out-of-Core).
* **TensorFlow** is a library for numerical computation using data flow graphs often used for AI.

#### Miscellaneous
* **scikit-image** provides tools for image processing.
* **SQLAlchemy** is an SQL toolkit and Object Relational Mapper.
* **SymPy** is library for symbolic mathematics, i.e. computer algebra system like Maple.
* **PyMC** features Bayesian statistical models including Markov chain Monte Carlo.
* **NetworkX** for working with structure, dynamics, and functions of complex networks.

<center><h1>Questions?</h1><br>
<img src=./gfx/light-bulb.jpg align="center" width=40%/></center>

# Credits

<br>
This presentation was inspired by:

* [Savarin's PyCon UK Tutorial](http://nbviewer.jupyter.org/github/savarin/pyconuk-introtutorial/blob/master/notebooks/Section%201-1%20-%20Filling-in%20Missing%20Values.ipynb)
* [Thomas Wiecki's Introduction to Data Analysis with Python](http://nbviewer.jupyter.org/github/twiecki/pydata_ninja/blob/master/PyData%20Ninja.ipynb)
* [Kaggle's Titantic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/)